In [1]:
import numpy as np
import numpy.linalg as lin
import scipy.stats as sts
import scipy.optimize as opt

In [2]:
data = np.loadtxt('/Users/rubyzhang/Desktop/UChicago/OSML/BootCamp2017/Econ/Wk4_StrEst/data/MacroSeries.txt',delimiter=',')
ct = data[:,0]
kt = data[:,1]
wt = data[:,2]
rt = data[:,3]

In [7]:
def data_moments(wt,kt,ct,rt,alpha,beta,rho,mu):
    zt = np.log(rt/(alpha*(kt**(alpha-1))))
    data_moms = np.empty(4)
    
    et = np.zeros(len(zt)-1)
    et = zt[1:]-rho*zt[:-1]-(1-rho)*mu
    data_moms[0] = np.mean(et)
    
    et2 = np.zeros(len(zt)-1)
    et2 = et*zt[:-1]
    data_moms[1] = np.mean(et2)
    
    mc1 = np.zeros(len(zt)-1)
    mc1 = (beta*alpha*np.exp(zt[1:])*(kt[1:]**(alpha-1))*ct[:-1]/ct[1:])-1
    data_moms[2] = np.mean(mc1)
    
    mc2 = np.zeros(len(zt)-1)
    mc2 = mc1*wt[:-1]
    data_moms[3] = np.mean(mc2)
    
    return data_moms

def err_vec(wt,kt,ct,rt,alpha,beta,rho,mu,simple):
    moms_data = data_moments(wt,kt,ct,rt,alpha,beta,rho,mu)
    moms_model = np.zeros_like(moms_data)
    if simple:
        err_vec = moms_model - moms_data
    else:
        err_vec = (moms_model - moms_data) / moms_data
    return err_vec

def criterion(params, *args):
    alpha,beta,rho,mu = params
    wt,kt,ct,rt = args
    W = np.eye(4)
    err = err_vec(wt,kt,ct,rt,alpha,beta,rho,mu,simple=True)
    crit_val = np.dot(np.dot(err.T, W), err) 
    return crit_val 

alpha_init = 0.46
beta_init = 0.5
rho_init = 0.72
mu_init = 9.5
params_init = np.array([alpha_init, beta_init, rho_init, mu_init])
args = (wt,kt,ct,rt)
bounds = ((1e-10,1-1e-10),(1e-10,1-1e-10),(-1+1e-10,1-1e-10),(1e-10,None))
results = opt.minimize(criterion, params_init, args=args, bounds=bounds)
alpha_MLE, beta_MLE, rho_MLE, mu_MLE= results.x
print(results)
print('alpha_MLE=', alpha_MLE, 'beta_MLE', beta_MLE, 'rho_MLE=', rho_MLE, 'mu_MLE=', mu_MLE)
print('Minimized Criterion:', criterion(results.x, *args))

      fun: 0.20907656503349464
 hess_inv: <4x4 LbfgsInvHessProduct with dtype=float64>
      jac: array([  4.21237237e+01,   4.69187433e+04,  -1.50721156e+00,
         2.38588895e+00])
  message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 90
      nit: 3
   status: 0
  success: True
        x: array([ 0.46,  0.99,  0.72,  9.5 ])
alpha_MLE= 0.460000000048 beta_MLE 0.989999995212 rho_MLE= 0.72 mu_MLE= 9.5
Minimized Criterion: 0.209076565033


In [4]:
alpha_init = 0.9
beta_init = 0.2
rho_init = -0.2
mu_init = 20
params_init = np.array([alpha_init, beta_init, rho_init, mu_init])
args = (wt,kt,ct,rt)
bounds = ((1e-10,1-1e-10),(1e-10,1-1e-10),(-1+1e-10,1-1e-10),(1e-10,None))
results = opt.minimize(criterion, params_init, args=args, bounds=bounds)
alpha_MLE, beta_MLE, rho_MLE, mu_MLE= results.x
print(results)
print('alpha_MLE=', alpha_MLE, 'beta_MLE', beta_MLE, 'rho_MLE=', rho_MLE, 'mu_MLE=', mu_MLE)
print('Minimized Criterion:', criterion(results.x, *args))

      fun: 1871.6198007732812
 hess_inv: <4x4 LbfgsInvHessProduct with dtype=float64>
      jac: array([-24229.75715035,  16441.56955081,  -3119.04757382,    204.54499463])
  message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 85
      nit: 3
   status: 0
  success: True
        x: array([  0.9 ,   0.99,  -0.2 ,  20.  ])
alpha_MLE= 0.900000003649 beta_MLE 0.989999995074 rho_MLE= -0.199999999986 mu_MLE= 20.0
Minimized Criterion: 1871.61980077


It appears that the estimated parameter values except for $\beta$ are extremely sensitive to initial starting values. The minimized criterion is found when using the answers obtained for problem 2 for $\alpha, \rho, \mu$.